# 1. 라이브러리 호출

In [79]:
import requests
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

import time
import re
import csv 
import glob
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import seaborn as sns

from bs4 import BeautifulSoup
from tqdm import tqdm

In [80]:
#지역, 식당 URL을 저장하기위한 딕셔너리
place_url_dic={}
#지역+식당 장소만을 저장하기 위한 배열
place_name_list=[]
#최종 배열
place_url_arr=[]

# 2. 식당별 url 추출

In [81]:
def extract_url():
    #식당의 수를 세기 위한 변수(최대 75개)
    num=1
    #동적 크롤링을 위한 크롬드라이버
    #driver = webdriver.Chrome(executable_path=r'D:\temp\chromedriver.exe')
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.implicitly_wait(6)

    #사용자로부터 원하는 지역 검색
    place = "배곧" 
    place = place.replace(' ', '') 
    place_url="https://m.map.naver.com/search2/search.naver?query={}맛집"

    #url접속
    driver.get(place_url.format(place))

    #식당 리스트를 가지고 있는 태그 추출

    #식당 이름 추출
    while num<=75:
        restaurant_name=driver.find_element_by_css_selector('#ct > div.search_listview._content._ctList > ul>li:nth-child({}) > div.item_info > a.a_item.a_item_distance._linkSiteview > div > strong'.format(num))
        restaurant_name_text=restaurant_name.text
        place_name_list.append(place+" "+restaurant_name_text)
        print(place+" "+restaurant_name_text)
        num+=1

    #개별 URL추출
    for i, keyword in enumerate(place_name_list):
        try:
            #검색 url 만들기
            naver_map_search_url = f'https://map.naver.com/v5/search/{keyword}/place' 
            driver.get(naver_map_search_url) #검색 url 접속, 즉 검색하기
            
            time.sleep(1) 
            cu = driver.current_url # 검색이 성공된 플레이스에 대한 개별 페이지
            
            res_code = re.findall(r"place/(\d+)", cu)
            final_url = 'https://pcmap.place.naver.com/restaurant/'+res_code[0]+'/review/visitor#'
            
            place_url_dic={'region':place, 'name':keyword,'url':final_url}
            place_url_arr.append(place_url_dic)
            print(place+" "+keyword+" : "+final_url)
        except IndexError:
            place_url_dic={'region':place, 'name':keyword,'url':'none'}
            place_url_arr.append(place_url_dic)
            print(place+" "+keyword+" : none")
    


## 2-(1) 식당별로 추출한 url csv 형태로 저장

In [82]:
def place_url_csv():
    with open('place_name_url.csv', 'w', newline='') as csv_file:
        fieldnames = ['region', 'name','url']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        for data in place_url_arr:
            writer.writerow(data)

## 2-(2) 추출한 url의 csv파일 불러오기

In [83]:
#추출한 url의 csv파일 불러오기
place_url_list=pd.read_csv('place_name_url.csv',encoding='utf-8')
place_url=place_url_list[['region','name','url']]

In [84]:
place_url.shape

(75, 3)

## 2-(3) url이 none인 값 제거

In [85]:
#url이 none인 값 제거
data=pd.DataFrame(place_url)
data2=data[data.url!='none']

# 3. 네이버 플레이스 방문자 리뷰 정보 추출

In [86]:
visitor_review_dic={}

#방문자 리뷰 정보를 담기 위한 최종 배열
visitor_review_arr=[]

In [87]:
#driver = webdriver.Chrome(executable_path=r'D:\temp\chromedriver.exe')
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.implicitly_wait(4)

for idx,row in data2.iterrows():
    try:
        #식당 정보 url, 이름, 지역 가져오기
        url=row['url']
        place_name=row['name']
        region=row['region']
        print('========================')
        print(place_name + '식당')

        #url접속
        driver.get(url)
        driver.implicitly_wait(3)

        #더보기 버튼 다 누르기

        for i in range(3): #리뷰 100개씩만 가져오기 - 시간관계상
            try:
                driver.find_element_by_tag_name('body').send_keys(Keys.END) #맨 밑으로 내리기
                driver.find_element_by_css_selector('#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section._3fSeV > div._2kAri > a').click()
                driver.find_element_by_tag_name('body').send_keys(Keys.END)
                time.sleep(1)
            except NoSuchElementException:
                print("-더보기 버튼 모두 클릭 완료-")
                break

        #파싱
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        #print(soup)

        #식당 리뷰 개수 출력
        reviews = soup.find_all('li', attrs = {'class':'_3l2Wz'}) #-> _3l2Wz 이 부분은 네이버에서 주기적으로 바꿈.
        review_num = len(reviews) #특정 식당의 리뷰 총 개수
        print('리뷰 총 개수 : ' + str(review_num))

        #넘버링
        num = []
        for i in range(0, review_num):
            num.append(i+1)

        #크롤링 작업
        category = '방문자 리뷰'
        for i in range(0, review_num):
            # review 내용
            try : 
                review_content = reviews[i].find('span', attrs = {'class':'WoYOw'}).text

            except: #리뷰가 없다면
                review_content = ""

            visitor_review_dic = {
                'num':num[i],
                'name' : place_name,
                'blog or review' : category,
                'visitor review content' : review_content
            }
            visitor_review_arr.append(visitor_review_dic)
        
    except TimeoutException:
        print("TimeoutException")
        
    except NoSuchElementException:
        print("NoSuchElementException")

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\jhu56\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


배곧 우정횟집식당
리뷰 총 개수 : 30
배곧 소쇄원 배곧본점식당


WebDriverException: Message: chrome not reachable
  (Session info: chrome=102.0.5005.63)


## 3-(1) 네이버 플레이스 방문자 리뷰 정보 csv 형태로 저장

In [ ]:
def place_review_csv():
    with open('place_visitor_review.csv','w',encoding='utf-8-sig') as csv_file:
        fieldnames = ['num', 'name','blog or review','visitor review content']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        for data in visitor_review_arr:
            writer.writerow(data)

# 3. 블로그 리뷰 정보 추출